# Overview

This notebook processes and visualizes the recreation data for the San Gabriel National Monument.

# Setup

In [108]:
# Import libraries
import arcpy, pandas as pd, numpy as np, arcgis, os
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [109]:
# Set up workspace
arcpy.env.workspace  = r"C:\Users\kathr\Documents\outdoor-alliance\mt-hood\mt-hood-analysis\mh_output.gdb"
arcpy.env.overwriteOutput = True

In [110]:
# Set gdb path
gdb_path = r"C:\Users\kathr\Documents\outdoor-alliance\mt-hood\mt-hood-analysis\mh_output.gdb"

# Trails

In [14]:
# Create pandas dataframe from feature class
trails_df = pd.DataFrame.spatial.from_featureclass("trails_clipped")

In [15]:
# View the dataframe
trails_df.head()

,OBJECTID,permanentidentifier,name,namealternate,trailnumber,trailnumberalternate,sourcefeatureid,sourcedatasetid,sourcedatadecscription,sourceoriginator,loaddate,trailtype,hikerpedestrian,bicycle,packsaddle,atv,motorcycle,ohvover50inches,snowshoe,crosscountryski,dogsled,snowmobile,nonmotorizedwatercraft,motorizedwatercraft,primarytrailmaintainer,nationaltraildesignation,lengthmiles,networklength,Length_Miles,SHAPE
0,1,29a6416a-d9b6-4ce8-8a3b-395bc17b44a3,Mt. Defiance Tie,None,413B,None,1572223010602,{FCB8819D-5F52-4371-91AC-879EB080369D},USFS Trails 12/2019,U.S. Forest Service,2022-01-19 06:48:08,Terra Trail,Y,None,None,None,None,None,None,None,None,None,None,None,FS,None,0.633631,11178.672020,0.095197,"{""hasZ"": true, ""hasM"": true, ""paths"": [[[-121...."
1,2,e9769317-8c36-43a1-9b12-42775f7b1b85,Bottle Prairie,EIGHTMILE LOOP,455,496,6487.004571,{FCB8819D-5F52-4371-91AC-879EB080369D},USFS Trails 12/2019,U.S. Forest Service,2022-01-19 06:48:08,Terra Trail,Y,None,None,None,None,None,None,None,None,None,None,None,FS,None,0.424867,80.352574,0.424868,"{""hasZ"": true, ""hasM"": true, ""paths"": [[[-121...."
2,3,431b2da6-2085-4080-b627-b58d3e3869dc,Big Hollow,None,158,None,5937.004571,{FCB8819D-5F52-4371-91AC-879EB080369D},USFS Trails 12/2019,U.S. Forest Service,2022-01-19 06:48:08,Terra Trail,Y,None,None,None,None,None,None,None,None,None,None,None,FS,None,3.067201,35.371833,0.021891,"{""hasZ"": true, ""hasM"": true, ""paths"": [[[-122...."
3,4,9d92d354-fe20-4149-b553-8c22431a633b,Three Mile South Fork,None,466.1,None,1872016010602,{FCB8819D-5F52-4371-91AC-879EB080369D},USFS Trails 12/2019,U.S. Forest Service,2022-01-19 06:48:08,Terra Trail,Y,None,None,None,None,None,None,None,None,None,None,None,FS,None,0.970692,78.035939,0.073422,"{""hasZ"": true, ""hasM"": true, ""paths"": [[[-121...."
4,5,0df1c82d-3120-49a6-ac0c-d01e3d1b7375,Tilly Jane Ski,None,643,SNO-643,5027.004901,{FCB8819D-5F52-4371-91AC-879EB080369D},USFS Trails 12/2019,U.S. Forest Service,2022-01-19 06:48:08,Terra Trail,Y,None,None,None,None,None,None,Y,None,None,None,None,FS,None,2.696591,11178.672020,2.696597,"{""hasZ"": true, ""hasM"": true, ""paths"": [[[-121...."


In [16]:
# Total number of trails - all uses
trails_df.shape[0]

990

In [20]:
# Total miles of trails - all uses
np.sum(trails_df["Length_Miles"])

1384.1604817618913

In [9]:
# Possible uses for trails:
# Bicycle
# Packsaddle
# ATV
# Motorcycle
# ohvover50inches
# Snowshoe
# XC Ski
# Dogsled
# Snowmobile
# Hiker/pedestrian

## Biking Trails

In [21]:
# Find possible values for bike column
trails_df["bicycle"].unique()

array([None, 'N', 'Y'], dtype=object)

In [22]:
# Subset to all trails that allow biking
mtb_trails = trails_df[trails_df["bicycle"] == "Y"]

In [25]:
# Total miles of trails - mtb (multi-use)
np.sum(mtb_trails["Length_Miles"])

36.72272904653071

In [26]:
# Total number of trails - mtb (multi-use)
mtb_trails.shape[0]

53

## Hiking Trails

In [27]:
# Find possible values for hiking column
trails_df["hikerpedestrian"].unique()

array(['Y', None], dtype=object)

In [28]:
# Subset to all trails that allow hiking
hiking_trails = trails_df[trails_df["hikerpedestrian"] == "Y"]

In [29]:
# Total miles of trails - hiking (multi-use)
np.sum(hiking_trails["Length_Miles"])

1337.790302250798

In [30]:
# Total number of trails - hiking (multi-use)
hiking_trails.shape[0]

977

In [31]:
# Subset to trails that are only hiking, no other activities
hike_only = hiking_trails[(hiking_trails["bicycle"] != "Y") & (hiking_trails["packsaddle"] != "Y") & (hiking_trails["atv"] != "Y") & (hiking_trails["motorcycle"] != "Y") & (hiking_trails["ohvover50inches"] != "Y") & (hiking_trails["dogsled"] != "Y") & (hiking_trails["snowmobile"] != "Y")]

In [32]:
# Total miles of trails - hiking (hiking only)
np.sum(hike_only["Length_Miles"])

1044.4332127510943

In [33]:
# Total number of trails - hiking (hiking only)
hike_only.shape[0]

782

## Equestrian Trails

In [34]:
# Subset to all trails that allow equestrian use
eq_trails = trails_df[trails_df["packsaddle"] == "Y"]

In [35]:
# Total miles of trails - equestrian (multi-use)
np.sum(eq_trails["Length_Miles"])

289.1529758873576

In [36]:
# Total number of trails - equestrian (multi-use)
eq_trails.shape[0]

194

# Paddling

## River Access Sites

In [42]:
# Create pandas dataframe from feature class
river_access_df = pd.DataFrame.spatial.from_featureclass("mh_river_access")

In [43]:
# Number of access sites
river_access_df.shape[0]

36

## Waterways

In [39]:
# Create pandas dataframe from feature class
paddle_df = pd.DataFrame.spatial.from_featureclass("mh_whitewater")

In [40]:
# Total number of miles of whitewater paddling
np.sum(paddle_df["Length_Miles"])

82.38676926037283

# Rock Climbing

In [30]:
# Create pandas dataframe from feature class
climbing_df = pd.DataFrame.spatial.from_featureclass("climbing_clipped")

In [31]:
# Total number of climbing sites
len(climbing_df["area_name"].unique())

164

# Picnic Areas

In [32]:
# Create pandas dataframe from feature class
picnic_df = pd.DataFrame.spatial.from_featureclass("mh_picnic")

In [33]:
# Total number of picnic sites
len(picnic_df["RECAREANAM"].unique())

9

# Camping

In [111]:
# Create pandas dataframe from feature class
camping_df = pd.DataFrame.spatial.from_featureclass("mh_camping")

In [112]:
# Total number of camping sites
len(camping_df["RECAREANAM"].unique())

40

# Impact of Expansion

## Current Rec

In [116]:
# Select AOI by attribute: Status = Designated
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-attribute.htm
in_layer = "mh_aoi"
selection_type = "NEW_SELECTION"
where_clause = "Status = 'Designated'"

aoi_designated = arcpy.management.SelectLayerByAttribute(in_layer_or_view = in_layer, 
                                        selection_type = selection_type, 
                                        where_clause = where_clause)

In [45]:
# Trails
arcpy.analysis.Clip("trails_clipped", aoi_designated, "mh_designated_trails")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\mh_designated_trails'>

In [46]:
# Recalculate length field after clipping
arcpy.management.CalculateGeometryAttributes(in_features = "mh_designated_trails",
                                             geometry_property = [["Length_Miles", "LENGTH_GEODESIC"]],
                                             length_unit = "MILES_INT")

<Result 'mh_designated_trails'>

In [47]:
# Paddling
arcpy.analysis.Clip("mh_whitewater", aoi_designated, "designated_whitewater")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\designated_whitewater'>

In [48]:
# Recalculate length field after clipping
arcpy.management.CalculateGeometryAttributes(in_features = "designated_whitewater",
                                             geometry_property = [["Length_Miles", "LENGTH"]],
                                             length_unit = "MILES_INT")

<Result 'designated_whitewater'>

In [49]:
in_layer = "mh_river_access"
overlap_type = "INTERSECT"
selection_type = "SUBSET_SELECTION"
selecting_features = aoi_designated

designated_river = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       selection_type = selection_type)
arcpy.management.CopyFeatures(designated_river, "designated_river")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\designated_river'>

In [50]:
# Climbing
in_layer = "climbing_clipped"
overlap_type = "INTERSECT"
selection_type = "SUBSET_SELECTION"
selecting_features = aoi_designated

designated_climbing = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                            overlap_type = overlap_type,
                                                            select_features = selecting_features,
                                                            selection_type = selection_type)
arcpy.management.CopyFeatures(designated_climbing, "designated_climbing")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\designated_climbing'>

In [117]:
# Picnicking
in_layer = "mh_picnic"
selection_type = "SUBSET_SELECTION"
overlap_type = "WITHIN_A_DISTANCE"
search_distance = "75 Meters"
selecting_features = aoi_designated

designated_picnic = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                           overlap_type = overlap_type,
                                                           search_distance = search_distance,
                                                           select_features = selecting_features,
                                                           selection_type = selection_type)
arcpy.management.CopyFeatures(designated_picnic, "designated_picnic")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\designated_picnic'>

In [118]:
# Camping
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-location.htm
in_layer = "mh_camping"
selection_type = "SUBSET_SELECTION"
overlap_type = "INTERSECT"
selecting_features = aoi_designated

designated_camping = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                           overlap_type = overlap_type, 
                                                            search_distance = search_distance,
                                                           select_features = selecting_features,
                                                           selection_type = selection_type)
arcpy.management.CopyFeatures(designated_camping, "designated_camping")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\designated_camping'>

## Expansion Rec

In [113]:
# Select AOI by attribute: Status = Proposed
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-attribute.htm
in_layer = "mh_aoi"
selection_type = "NEW_SELECTION"
where_clause = "Status = 'Proposed'"

aoi_proposed = arcpy.management.SelectLayerByAttribute(in_layer_or_view = in_layer, 
                                        selection_type = selection_type, 
                                        where_clause = where_clause)

In [54]:
# Trails
arcpy.analysis.Clip("trails_clipped", aoi_proposed, "proposed_trails")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\proposed_trails'>

In [55]:
# Recalculate length field after clipping
arcpy.management.CalculateGeometryAttributes(in_features = "proposed_trails",
                                             geometry_property = [["Length_Miles", "LENGTH_GEODESIC"]],
                                             length_unit = "MILES_INT")

<Result 'proposed_trails'>

In [56]:
# Paddling
arcpy.analysis.Clip("mh_whitewater", aoi_proposed, "proposed_whitewater")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\proposed_whitewater'>

In [57]:
# Recalculate length field after clipping
arcpy.management.CalculateGeometryAttributes(in_features = "proposed_whitewater",
                                             geometry_property = [["Length_Miles", "LENGTH"]],
                                             length_unit = "MILES_INT")

<Result 'proposed_whitewater'>

In [58]:
in_layer = "mh_river_access"
overlap_type = "INTERSECT"
selection_type = "SUBSET_SELECTION"
selecting_features = aoi_proposed

proposed_river = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       selection_type = selection_type)
arcpy.management.CopyFeatures(proposed_river, "proposed_river")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\proposed_river'>

In [59]:
# Climbing
in_layer = "climbing_clipped"
overlap_type = "INTERSECT"
selection_type = "SUBSET_SELECTION"
selecting_features = aoi_proposed

proposed_climbing = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       selection_type = selection_type)
arcpy.management.CopyFeatures(proposed_climbing, "proposed_climbing")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\proposed_climbing'>

In [114]:
# Picnicking
in_layer = "mh_picnic"
overlap_type = "INTERSECT"
selection_type = "SUBSET_SELECTION"
selecting_features = aoi_proposed

proposed_picnic = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       selection_type = selection_type)
arcpy.management.CopyFeatures(proposed_picnic, "proposed_picnic")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\proposed_picnic'>

In [115]:
# Camping
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-location.htm
in_layer = "mh_camping"
overlap_type = "INTERSECT"
selection_type = "SUBSET_SELECTION"
selecting_features = aoi_proposed

proposed_camping = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       selection_type = selection_type)
arcpy.management.CopyFeatures(proposed_camping, "proposed_camping")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\proposed_camping'>

## Comparison

In [67]:
# Define function to do comparison
def compare(designated, proposed):
    return (proposed / designated) * 100

### Trails

In [62]:
# Trails
designated_trails = pd.DataFrame.spatial.from_featureclass("mh_designated_trails")
proposed_trails = pd.DataFrame.spatial.from_featureclass("proposed_trails")

In [63]:
# Hiking multiuse
hiking_trails_d = designated_trails[designated_trails["hikerpedestrian"] == "Y"]
print(hiking_trails_d.shape[0])

hiking_trails_p = proposed_trails[proposed_trails["hikerpedestrian"] == "Y"]
print(hiking_trails_p.shape[0])

709
361


In [64]:
# Total miles of trails multiuse
print(np.sum(hiking_trails_d["Length_Miles"]))
print(np.sum(hiking_trails_p["Length_Miles"]))

984.2673556245902
352.3032555055467


In [68]:
# % increase
compare(np.sum(hiking_trails_d["Length_Miles"]), np.sum(hiking_trails_p["Length_Miles"]))

35.79345118908107

In [69]:
# Hiking only designated
hike_only_d = hiking_trails_d[(hiking_trails_d["bicycle"] != "Y") & (hiking_trails_d["packsaddle"] != "Y") & (hiking_trails_d["atv"] != "Y") & (hiking_trails_d["motorcycle"] != "Y") & (hiking_trails_d["ohvover50inches"] != "Y") & (hiking_trails_d["dogsled"] != "Y") & (hiking_trails_d["snowmobile"] != "Y")]
print(np.sum(hike_only_d["Length_Miles"]))
print(hike_only_d.shape[0])

# Hiking only proposed
hike_only_p = hiking_trails_p[(hiking_trails_p["bicycle"] != "Y") & (hiking_trails_p["packsaddle"] != "Y") & (hiking_trails_p["atv"] != "Y") & (hiking_trails_p["motorcycle"] != "Y") & (hiking_trails_p["ohvover50inches"] != "Y") & (hiking_trails_p["dogsled"] != "Y") & (hiking_trails_p["snowmobile"] != "Y")]
print(np.sum(hike_only_p["Length_Miles"]))
print(hike_only_p.shape[0])

706.4269312724043
544
336.78659048160085
331


In [70]:
# % increase
compare(np.sum(hike_only_d["Length_Miles"]), np.sum(hike_only_p["Length_Miles"]))

47.674653325431194

In [71]:
# MTB designated
mtb_trails_d = designated_trails[designated_trails["bicycle"] == "Y"]
print(np.sum(mtb_trails_d["Length_Miles"]))
print(mtb_trails_d.shape[0])

# MTB proposed
mtb_trails_p = proposed_trails[proposed_trails["bicycle"] == "Y"]
print(np.sum(mtb_trails_p["Length_Miles"]))
print(mtb_trails_p.shape[0])

21.73446385151827
24
14.988265195012442
29


In [72]:
# % increase
compare(np.sum(mtb_trails_d["Length_Miles"]), np.sum(mtb_trails_p["Length_Miles"]))

68.96082322254031

In [73]:
# Equestrian proposed
eq_trails_d = designated_trails[designated_trails["packsaddle"] == "Y"]
print(np.sum(eq_trails_d["Length_Miles"]))
print(eq_trails_d.shape[0])

# Equestrian proposed
eq_trails_p = proposed_trails[proposed_trails["packsaddle"] == "Y"]
print(np.sum(eq_trails_p["Length_Miles"]))
print(eq_trails_p.shape[0])

277.84042435218583
165
11.312551411599708
29


In [48]:
# % increase
compare(np.sum(eq_trails_d["Length_Miles"]), np.sum(eq_trails_p["Length_Miles"]))

4.071600249667092

### Waterways

In [74]:
# Whitewater paddling
designated_whitewater = pd.DataFrame.spatial.from_featureclass("designated_whitewater")
proposed_whitewater = pd.DataFrame.spatial.from_featureclass("proposed_whitewater")

In [76]:
print(np.sum(designated_whitewater["Length_Miles"]))
print(np.sum(proposed_whitewater["Length_Miles"]))

42.1917419397345
40.18922250947508


In [77]:
# % increase
compare(np.sum(designated_whitewater["Length_Miles"]), np.sum(proposed_whitewater["Length_Miles"]))

95.25376450889426

In [79]:
# River access
designated_river = pd.DataFrame.spatial.from_featureclass("designated_river")
proposed_river = pd.DataFrame.spatial.from_featureclass("proposed_river")

In [80]:
print(designated_river.shape[0])
print(proposed_river.shape[0])

15
0


In [54]:
# % increase
compare(designated_river.shape[0], proposed_river.shape[0])

133.33333333333331

### Climbing

In [55]:
# Climbing
designated_climbing = pd.DataFrame.spatial.from_featureclass("designated_climbing")
proposed_climbing = pd.DataFrame.spatial.from_featureclass("proposed_climbing")

In [56]:
print(len(designated_climbing["area_name"].unique()))
print(len(proposed_climbing["area_name"].unique()))

103
61


In [57]:
# % increase
compare(len(designated_climbing["area_name"].unique()), len(proposed_climbing["area_name"].unique()))

59.22330097087378

### Picnic

In [71]:
# Picnic
designated_picnic = pd.DataFrame.spatial.from_featureclass("designated_picnic")
proposed_picnic = pd.DataFrame.spatial.from_featureclass("proposed_picnic")

In [72]:
print(len(designated_picnic["RECAREANAM"].unique()))
print(len(proposed_picnic["RECAREANAM"].unique()))

2
5


In [73]:
# % increase
# manually inspected map - there are 2 picnic areas in designated spots that are just a bit outside of the boundary due to gps inaccuracy
compare(4, len(proposed_picnic["RECAREANAM"].unique()))

125.0

### Camping

In [119]:
# Camping
designated_camping = pd.DataFrame.spatial.from_featureclass("designated_camping")
proposed_camping = pd.DataFrame.spatial.from_featureclass("proposed_camping")

In [120]:
print(len(designated_camping["RECAREANAM"].unique()))
print(len(proposed_camping["RECAREANAM"].unique()))

1
39


In [121]:
# % increase
compare(len(designated_camping["RECAREANAM"].unique()), len(proposed_camping["RECAREANAM"].unique()))

3900.0

# Clustering Analysis

In [34]:
# Get point features for the start/end of trails
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/feature-vertices-to-points.htm
arcpy.management.FeatureVerticesToPoints(in_features = "trails_clipped",
                                         out_feature_class = "trail_points",
                                         point_location = "START")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\trail_points'>

In [36]:
# Merge all the rec points together into one layer
# Define layers
rec_sites = os.path.join(gdb_path, r"Recreation_Sites\Recreation_Sites")
mh_picnic = os.path.join(gdb_path, "mh_picnic")
mh_camping = os.path.join(gdb_path, "mh_camping")
river_access = os.path.join(gdb_path, r"River_Access\River_Access")
climbing = os.path.join(gdb_path, "climbing_clipped")

In [38]:
# Create field mapping object
field_mappings = arcpy.FieldMappings()

# Add input field for rec area name into new output field
map_name = arcpy.FieldMap()
map_name.addInputField(rec_sites, "NAME")
map_name.addInputField(climbing, "area_name")
map_name.addInputField(river_access, "name")
map_name.addInputField(mh_picnic, "RECAREANAM")
map_name.addInputField(mh_camping, "RECAREANAM")

# Set name of new output field for site name
rec_name = map_name.outputField
rec_name.name = "Rec_Name"
map_name.outputField = rec_name

# Add output fields to field mappings object
field_mappings.addFieldMap(map_name)

In [39]:
# Merge
arcpy.management.Merge([rec_sites, river_access, climbing, mh_picnic, mh_camping], 
                       "all_rec", 
                       field_mappings,
                      add_source = "ADD_SOURCE_INFO")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\all_rec'>

In [ ]:
# Run the Density-Based Clustering analysis
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-statistics/densitybasedclustering.htm

arcpy.stats.DensityBasedClustering(in_features = "all_rec", 
                                   output_features = "rec_density", 
                                   cluster_method = "HDBSCAN", # TODO try OPTICS
                                   min_features_cluster = 10)

# Maps

## Map all recreation types

In [247]:
# Set up environment
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprx.defaultGeodatabase = gdb_path

In [249]:
# all_rec = aprx.createMap(name = "Recreation",
#                         map_type = "MAP")
all_rec = aprx.listMaps("Recreation")[0] 

In [250]:
# Add recreation data
all_rec.addDataFromPath(os.path.join(gdb_path, "mh_aoi"))
all_rec.addDataFromPath(os.path.join(gdb_path, "trails_clipped"))
all_rec.addDataFromPath(os.path.join(gdb_path, "mh_whitewater"))
all_rec.addDataFromPath(os.path.join(gdb_path, "climbing_clipped"))
all_rec.addDataFromPath(os.path.join(gdb_path, "mh_picnic"))
all_rec.addDataFromPath(os.path.join(gdb_path, "mh_camping"))

In [251]:
aoi = all_rec.listLayers("mh_aoi")[0]
trails = all_rec.listLayers("trails_clipped")[0]
paddle = all_rec.listLayers("mh_whitewater")[0]
picnic = all_rec.listLayers("mh_picnic")[0]
camp = all_rec.listLayers("mh_camping")[0]
climb = all_rec.listLayers("climbing_clipped")[0]


all_lyr = [trails, paddle, picnic, camp, climb, aoi]

In [264]:
for lyr in all_lyr:
    sym = lyr.symbology
    # Update symbology for recreation types
    if(lyr.name == "trails_clipped"):
        sym.renderer.symbol.color = {"RGB": [0, 0, 0, 100]}
    elif(lyr.name == "mh_whitewater"):
        sym.renderer.symbol.color = {"RGB": [0, 92, 230, 100]}
    elif(lyr.name == "mh_picnic"):
        sym.renderer.symbol.color = {"RGB": [255, 0, 197, 100]}
    elif(lyr.name == "mh_camping"):
        sym.renderer.symbol.color = {"RGB": [58, 45, 168, 100]}
    elif(lyr.name == "climbing_clipped"):
        sym.renderer.symbol.color = {"RGB": [166, 216, 84, 100]}
    
    # Update symbology for AOI polygons
    elif(lyr.name == "mh_aoi"):
        sym.updateRenderer("UniqueValueRenderer")
        #sym.renderer.fields = ["Layer"]
        #sym.renderer.colorRamp = aprx.listColorRamps("Set 2 (3 classes)")[0]
        for grp in sym.renderer.groups:
            for itm in grp.items:
                myVal = itm.values[0][0]
                #print("myVal: {0}".format(myVal))
                if "Prop" in myVal:
                    itm.symbol.color = {"RGB": [252, 141, 98, 100]}
                else: 
                    itm.symbol.color = {"RGB": [102, 194, 164, 100]}
        
    lyr.symbology = sym